# Last time we looked a predicting District matches from the regional events. Let's add some more features to our model.

Load the training data into a pandas data frame.  

In [17]:
from __future__ import print_function
import pickle
#import time
import sys
sys.path.append('..')
import swagger_client as v3client
from swagger_client.rest import ApiException

filename = 'matches_2019.pkl'
matches = []
with open(filename, 'rb') as f:
    matches = pickle.load(f)


Preview a match object

In [44]:
non_empty = [k for k in matches['matches'].keys() if len(matches['matches'][k])>0]
data = [m for k in matches['matches'] for m in matches['matches'][k]]
data = [m for m in data if m.winning_alliance!='']
print(len(data))
red = [x for m in data for x in m.alliances.red.team_keys]
blue = [x for m in data for x in m.alliances.blue.team_keys]
from collections import Counter
Counter(red+blue)

qualifiers = [m for m in data if m.comp_level=='qm']
finals = [m for m in data if m.comp_level!='qm']
alliances = [' '.join(m.alliances.red.team_keys) for m in qualifiers]  + [' '.join(m.alliances.blue.team_keys) for m in qualifiers]
alliances_finals = [' '.join(m.alliances.red.team_keys) for m in finals]  + [' '.join(m.alliances.blue.team_keys) for m in finals]
scores = [m.alliances.red.score for m in qualifiers] + [m.alliances.blue.score for m in qualifiers]
rp = [m.score_breakdown and m.score_breakdown['red']['rp'] or 0.0 for m in qualifiers] + [m.score_breakdown and m.score_breakdown['blue']['rp'] or 0.0 for m in qualifiers]
#foul = [m.alliances.red.]
scores_finals = [m.alliances.red.score for m in finals] + [m.alliances.blue.score for m in finals]
len(scores),len(scores_finals)  # should be 2x number of qualifiers

21240


(34314, 8166)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
import numpy as np
#from scipy.sparse.linalg import spsolve

# two count vectorizers. This transforms the alliance lists into vector encodings
vectorizer = CountVectorizer(max_features=15000, min_df=1, max_df=1.0, stop_words=None)  

X = vectorizer.fit_transform(alliances)
X_finals = vectorizer.transform(alliances_finals)
# shuffle the data first
#train = train.sample(frac=1.0)

# produce the training features and labels.
#X = ct.fit_transform(train)
y = np.column_stack([scores, rp])
y_finals = scores_finals

#X[1,:].todense()
np.shape(y)


(34314, 2)

In [48]:
from scipy.sparse.linalg import lsqr

opr = lsqr(X,y[:,0])[0]
opr_rp = lsqr(X,y[:,1])[0]
opr_all = np.column_stack([opr, opr_rp])
#print(opr)
opr_lookup=dict(zip(sorted(vectorizer.vocabulary_,key=lambda x: vectorizer.vocabulary_[x]),opr_all))
#for x,y in list(zip(list(X_finals*opr),y_finals)):
#    print('{}\t{}'.format(x,y))
#print(vectorizer.vocabulary_)
#list(sorted(vectorizer.vocabulary_,key=lambda x: vectorizer.vocabulary_[x]))
#print(X[0,:].todense())
#print(alliances[0])
#print(vectorizer.vocabulary_['frc5813'])
#print(vectorizer.vocabulary_['frc2342'])
#print(vectorizer.vocabulary_['frc3323'])
print(opr_lookup['frc492'])

[28.30255665  0.83450885]


In [49]:
teamAggregates = {}

def addMatch(team, m):    
    if team not in teamAggregates:       
        # initialize an empty record for the team.
        teamAggregates[team]= {
            'totalMatches':0, 
            'rpTotal': 0,
            'totalPoints': 0,
            'winCount': 0
        }
            

    alliance = 'blue' if team in m.alliances.blue.team_keys else 'red'
    points = m.score_breakdown[alliance]
    summary = teamAggregates[team]
    # update all of the fields.
    summary['totalMatches']+=1    
    
    summary['rpTotal'] += points['rp']
    summary['totalPoints'] += points['totalPoints']
    summary['winCount'] += int(m.winning_alliance==alliance)

# Currently just using Week0 events.
for m in qualifiers:
    if m.score_breakdown is None:
        continue
    # only aggregate statistics for regional matches- skip the districts or we're cheating.
    #if m.event_key=='2020pncmp':
    #    continue
    for t in m.alliances.red.team_keys:
        addMatch(t,m)
    for t in m.alliances.blue.team_keys:
        addMatch(t,m)
        
# normalize the aggregates
for t in teamAggregates:
    for k in teamAggregates[t]:
        if k=='totalMatches':
            continue
        teamAggregates[t][k]/=teamAggregates[t]['totalMatches']
    if t in opr_lookup:
        teamAggregates[t]['opr']=opr_lookup[t][0]
        teamAggregates[t]['opr_rp']=opr_lookup[t][1]
    else:
        print(t)

with open('teamStats_tmp_2019.pkl','wb') as f:
    pickle.dump(teamAggregates,f)

frc1574B
frc3211B
frc1690B
frc3339B
frc503B
frc3538B
frc4130B
frc27B
frc2834B
frc5687B
frc4564B
frc6328B
frc7127B
frc862B
frc33B
frc3604B
frc3707B
frc3655B
frc1023B
frc3641B
frc1678B
frc1678C
frc1323B
frc971B
frc5104B
frc2135B
frc973C
frc5458B
frc973B
frc254B
frc6465B
frc5817B
frc1671B
frc971C
frc6174B
frc299B
frc6657B
frc2601B
frc102B


In [52]:
with open('stats_tmp.tsv','w',encoding='utf-8') as outStats:
    outStats.write('{}\n'.format('\t'.join(['team','opr','opr_rp','rp','points','wins'])))
    for t in teamAggregates:
        k = teamAggregates[t]
        if 'opr' not in k:
            continue
        outStats.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format(t,k['opr'],k['opr_rp'],k['rpTotal'],k['totalPoints'],k['winCount']))